In [9]:
## pip install rdflib
from rdflib import Graph, URIRef, Namespace, Literal, BNode
from rdflib.namespace import RDF, FOAF
AGS_PORT = "https://portaal-stadantwerpen.opendata.arcgis.com/data.jsonld"
AOD_LIC = "https://www.antwerpen.be/info/5c9c967180779f42723823c9/gratis-open-data-licentie"
CONTACT = "gis@antwerpen.be"
HOME_URL = "https://antwerpen.be"

In [10]:
def dcat_theme(keywords=[]):
    "Find theme using keywords"
    for key in keywords:
        if key.lower() == "economie":
            return Literal('Economie') #URIRef("http://vocab.belgif.be/auth/datatheme/ECON")
        elif key.lower() in ["cultuur", "sport" ]:
            return Literal('Cultuur') #URIRef("http://vocab.belgif.be/auth/datatheme/CULT")
        elif key.lower() == "onderwijs":
            return Literal('Onderwijs') #URIRef("http://vocab.belgif.be/auth/datatheme/EDUC")
        elif key.lower() in ["afval", "bodem", "milieu", "groeninventaris"]:
            return Literal('Milieu') #URIRef("http://vocab.belgif.be/auth/datatheme/ENVI")
        elif key.lower() in ["publieke diensten"]:
            return Literal('Overheid') #URIRef("http://vocab.belgif.be/auth/datatheme/GOVE")
        elif key.lower() == "zorg":
            return Literal('Gezondheidszorg') #URIRef("http://vocab.belgif.be/auth/datatheme/HEAL")
        elif key.lower() in ["grenzen", "structuurplan"]:
            return Literal('Grenzen') #URIRef("http://vocab.belgif.be/auth/datatheme/REGI")
        elif key.lower() == "mobiliteit":
            return Literal('Transport en Mobiliteit') #URIRef("http://vocab.belgif.be/auth/datatheme/TRAN")   
    return Literal('geospatial')

In [11]:
## read graph and namespaces
g = Graph()
g.parse(AGS_PORT, format='json-ld')
dc = Namespace( "http://purl.org/dc/terms/" )
dcat = Namespace("http://www.w3.org/ns/dcat#")
g.namespace_manager.bind("foaf", FOAF)
g.namespace_manager.bind("dcat", URIRef("http://www.w3.org/ns/dcat#"))
g.namespace_manager.bind("dc", URIRef("http://purl.org/dc/terms/") )

In [12]:
## add license
g.add( [URIRef(AOD_LIC), RDF.type, dc.LicenseDocument ] ) 
g.add( [URIRef(AOD_LIC), dc.type, Literal("Public domain") ] ) 
g.add( (URIRef(HOME_URL), RDF.type, FOAF.Agent ) )
g.add( (URIRef(HOME_URL), FOAF.name, Literal('Stad Antwerpen') ) )

<Graph identifier=Nad9f36b22e75456cb32f0f612a1db19c (<class 'rdflib.graph.Graph'>)>

In [13]:
## add extra properties of the catalog
s2uri = URIRef("https://s3-ant1.antwerpen.be/prd-open-data-datatank/DCAT/dcat.xml")

for s, p, o in g.triples(( None,  RDF.type, dcat.Catalog )):
    
    g.add( (s2uri, p, o))
    for s2, p2, o2 in g.triples(( s,  None, None )):
        g.add( (s2uri, p2, o2))
    g.remove((s, None, None))
    
    g.add( (s2uri, dc.title, Literal("Opendata portaal Antwerpen")  ) )
    g.add( (s2uri, dc.description, Literal("""De stad Antwerpen beschikt over een schat aan geografische data over de stad. 
Via dit portaal wordt u de mogelijkheid geboden om deze geografische data te verkennen, 
visualiseren en te downloaden. Deze data is open data en mag u vrij en kosteloos gebruiken. 
Met de datasets kunt u bijvoorbeeld nieuwe mobiele toepassingen ontwikkelen die de dienstverlening voor bewoners kunnen verbeteren. 
Ontdek de data door hieronder te zoeken op de verschillende thema’s of door een trefwoord in te vullen.""") ) )
    g.add( (s2uri , dc.publisher , Literal("Stad Antwerpen")  ) )
    g.add( (s2uri , dc.license , URIRef(AOD_LIC)  ) )
    g.add( (s2uri , dc.language , URIRef("http://id.loc.gov/vocabulary/iso639-1/nl")  ) )
    

In [14]:
# ## add extra properties of the Datasets
# for s, p, o in g.triples(( None, RDF.type, dcat.Dataset)):
#     keyw = [str(t[2]) for t in g.triples(( s,  dcat.keyword, None))]
#     uri = g.value(s, dc.identifier)
#     g.add( (uri, RDF.type, dcat.Dataset))
#     g.add( (uri, dc.language , URIRef("http://id.loc.gov/vocabulary/iso639-1/nl") ) )
#     g.add( (uri, dcat.theme, dcat_theme(keyw) ))
    
#     for s2, p2, o2 in g.triples(( s, None , None )):
#         g.add( (uri, p2, o2))

#     g.set( (uri , dc.license , URIRef(AOD_LIC)  ) )

#     g.add(( s2uri,  dcat.dataset , uri  ))
#     g.remove((s2uri,  dcat.dataset , s ))
#     g.remove((s, None, None))
# print(keyw, uri)

In [15]:
## write xml
n3 = g.serialize(format='application/rdf+xml', indent=4)
with open("dcat.xml", 'w', encoding='utf-8' ) as f:
    f.write(n3)